# Text Retrieval and Mining - BSc BAN Y3 - Assignment

* BSc Business Analytics
* Faculty of Economics and Business
* University of Amsterdam
* Authors: Claudia Orellana Rodriguez, [Julien Rossi](mailto://j.rossi@uva.nl)

# Student Group

* Student 1
* Student 2
* Student 3
* Student 4

# Instructions

* This assignment is a group assignment
* Submit this notebook through Canvas
* The assignment is made of 3 questions
* The assignment includes an oral presentation in which you will describe your work and justify your answers
* For the coding in the assignment you are free to re-use all the course material
  * Lecture notebooks
  * Tutorial notebooks

___


**GRADING**
* Question 1: 5 points
* Question 2: 10 points
* Question 3: 5 points
* Presentation: 5 points


**GPU WARNING**
* We will use transformers
* It is best to use a GPU-based runtime on Google Colab (this notebook has the good runtime)
* With a powerful laptop you could achieve the same result, be careful with overheating during the encoding phase (Question 2)

# Data Loading

In [ ]:
import logging

logger = logging.getLogger("TRTM")
logger.setLevel(logging.INFO)

In [ ]:
import requests

r = requests.get("https://raw.githubusercontent.com/j-rossi-nl/teaching-data/main/2024_TRTM/climate_flying_text.zip")
assert r.status_code == 200

In [ ]:
import re

from io import BytesIO
from zipfile import ZipFile
from zipfile import ZipInfo

import pandas as pd

datafile_re = re.compile(r"^climate_flying_text/stayingOnTheGround_(?P<theme>[^_\.]+)(_text)?\.csv$")
dfs: list[pd.DataFrame] = []

with ZipFile(BytesIO(r.content), mode="r") as zf:
    info: ZipInfo
    for info in zf.filelist:
        if info.is_dir():
            continue

        m = datafile_re.match(info.filename)
        if m is None:
            logger.warning(f"Skipping {info.filename}")
            continue

        logger.info(f"Loading data from {info.filename}")
        with zf.open(info, mode="r") as datafile:
            df = pd.read_csv(datafile)
        df["theme"] = m.group("theme")
        dfs.append(df)

flying = pd.concat(dfs).drop(columns=["id"]).dropna(subset=["text"]).reset_index(drop=True)


In [ ]:
import spacy

In [ ]:
r = requests.get("https://surfdrive.surf.nl/files/index.php/s/Z32zhlJz5iuHEGu/download")
assert r.status_code == 200

In [ ]:
from spacy.tokens import DocBin
docbin = DocBin().from_bytes(r.content)

In [ ]:
nlp = spacy.load("en_core_web_sm")
flying["spacy"] = list(docbin.get_docs(vocab=nlp.vocab))

In [ ]:
flying.shape   # should be (129091, 3)

In [ ]:
flying.drop_duplicates(subset=["text"], inplace=True)

In [ ]:
flying.shape  # should be (40695, 3)

In [ ]:
import plotly.express as px

px.histogram(
    data_frame=flying,
    x="theme",
    color="theme"
)

# Data Description

You are working with a dataset of tweets collected under the umbrella "Staying on the ground", in reference to a swedish campaign ["Stay on the ground"](https://westayontheground.org/), against the continued growth of the aviation sector, in light of climate change.

This dataset is articulated around themes:
* `"equity"`: how are the changes equitable for everyone ?
* `"policySupportingAviationReduction"` and `"policySupportingAviationReduction2"`: about public policies that would reduce the aviation industry
* `"ccConsequences"`: about the consequences of climate change
* `"flightShame"`: shaming those who decide to fly
* `"alternatives`": better travel choices than flying
* `"practicingChange`": examples of people's practices

___


The dataframe `flying` has 3 columns:
* `text`: the original message
* `theme`: the theme of the message
* `spacy`: a `Doc` object from `spacy` corresponding to the message

___


In this assignment, you will study different facets of the dataset while practicing the following class material:
* Entities, POS tagging
* Topic Modeling
* Text Representation and Clustering

You do not need to process the dataset yourself with `SpaCy`, this is already done.

# Question 1 - Entities


**ASSIGNMENT**
* What are the TOP-3 most popular entity categories by number of mentions ?
* Who are the TOP-10 most popular persons ?
* Who are the TOP-10 most mentioned locations / geo-political entities ?
* What are the TOP-10 entity categories by number of mentions containing `"greta"`


# Question 2 - Clustering

We focus on the theme `"alternatives"`. Within this theme we want to cluster the messages, in a way similar to the topic modeling, without using LDA or BERTopic.

* The `"alternatives"` theme is made of 4913 messages
* We will create document embeddings
* Then cluster these embeddings using K-Means
* Then represent our clustering on a 2D scatter plot, which will require a dimensionality reduction

___


**DOCUMENT EMBEDDINGS**
* We will use [Sentence Transformers](https://huggingface.co/sentence-transformers) from huggingface
* The model is `"paraphrase-distilroberta-base-v1"`
* Instantiate the model
* Use it for encoding the messages
* The result is a `numpy.array` with shape `(4913, d)` where `d` is the number of dimensions of the document embeddings

___


**K-Means**
* Use [SKLearn KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)
* Use the elbow method
* Plot the inertia of each clustering model for a number of clusters from 1 to 20

___


**DIMENSIONALITY REDUCTION**
* We will use [UMAP](https://umap-learn.readthedocs.io/en/latest/basic_usage.html)
* Create a 2-D representation of the document embeddings
* Plot this 2-D representation as a scatter plot
* Each document is one dot, with a color representing the cluster it has been allocated to

___


**CLUSTERING ANALYSIS**
* Select 2 clusters
* Show for each cluster the 10 documents which have the embeddings the closest to the cluster centroid




In [ ]:
!pip install sentence-transformers umap-learn

# Question 3 - LDA

* Use the complete corpus
* We will tokenize the corpus
* Create a LDA model with 8 topics

___


**TOKENIZATION**
* Consider the lowercased corpus
* Use the `TweetTokenizer` from NLTK
* For tokenization, consider the following:
  * Remove the english stopwords (from the NLTK list of stopwords)
  * Consider `"rt"` as a stopword
  * Keep only the tokens made of letters only, at least 3 letters
* Create a `gensim` Dictionary
  * Filter out tokens that appear in less than 5 documents
  * Filter out tokens that appear in more than 90% of the documents


___


**LDA**
* Create a LDAModel from gensim
* With 8 topics
* Display the word clouds corresponding to the Top-10 words in each topic



